*The code snippet assumes Anaconda 5.2.0 version of Python virtual environment*

<div class="alert alert-info">
    <h4>Acknowledgement</h4>
    <p>The materials on this post are based the on four NLP papers, <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf" target="_blank">Distributed Representations of Words and Phrases and their Compositionality</a> (Mikolov et al., 2013), <a href="https://arxiv.org/pdf/1411.2738.pdf" target="_blank">word2vec Parameter Learning Explained</a> (Rong, 2014), <a href="https://arxiv.org/pdf/1710.09805.pdf" target="_blank">Improving Negative Sampling for Word Representation using Self-embedded Features</a> (Chen et al., 2017) and <a href="https://arxiv.org/pdf/1402.3722.pdf" target="_blank">word2vec Explained: Deriving Mikolov et al.’s
Negative-Sampling Word-Embedding Method</a> (Goldberg and Levy, 2014).</p>
</div>

## Review on Word2Vec Skip-Gram

In my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#eq-11" target="_blank">previous post</a>, I illustrated the neural network structure of Skip-Gram Word2Vec model that represents words in a vector space.

<div class="row" id="fig1">
    <div class="col"><img src="jupyter_images/word2vec_skip-gram.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 1: Skip-Gram model structure</p></div>
</div>

I also derived the cost function of Skip-Gram in <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#Derivation-of-Cost-Function">Derivation of Cost Function</a>:

<div id="eq-1" style="font-size: 1rem;">
$$J(\theta) = -
\frac{1}{T}
\sum_{t=1}^{T}
\sum_{c=1}^{C}
log 
\frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{1}$$
</div>

where $T$ is the size of training samples, $C$ is the <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#Window-Size-of-Skip-Gram">window size</a>, $V$ is the size of unique vocab in the corpus, and $W_{input}$, $W_{output}$ and $h$ are illustrated in <a href="#fig1">figure 1</a>. I also noted that <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic">stochastic gradient descent</a> (SGD) is used to mitigate computational burden — the size of $T$ in $\frac{1}{T} \sum^T_{t=1}$ can be billions or more in NLP applications. The new cost function using SGD is:

<div id="eq-2" style="font-size: 1rem;">
$$J(\theta; w^{(t)}) = -
\sum_{c=1}^{C}
log 
\frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{2}$$
</div>

### Review on Softmax

Softmax is a multinomial regression classifier. It means that it classifies multiple labels, such as predicting if an hand-written digit is $0,\,1,\,2,\,...\,8\,$ or $9$. In case of binary classification (True or False), such as classifying fraud or not-fraud in bank transactions, binomial regression classifier called Sigmoid function is used.

In <a href="#eq-2">eq (2)</a>, the fraction inside the summation of log yields the probability distribution of all $V$-vocabs in the corpus, given the input word. In statistics, the conditional probability of $A$ given $B$ is denoted as $p(A|B)$. In Skip-Gram, we use the notation, $p(w_{context}| w_{center})$, to denote the conditional probability of observing a context word given a center word. It is obtained by using the softmax function:

<div id="eq-3" style="font-size: 1rem;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{1} \tag{3} $$</div>

Exponentiation ensures that the transformed values are positive, and the normalization factor in the denominator ensures that the values have a range of $[0, 1)$ and their sum equals $1$. 

<div id="fig2" class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/softmax_function.png" style="height: 300px;"></div>
    <div class="col-12"><p class="image-description">Figure 2: softmax function transformation</p></div>
</div>

The probability is computed $V$ times using <a href="#eq-3">eq (3)</a> to obtain a conditional probability distribution of observing all $V$-unique vocabs in the corpus, given a center word ($w^{(t)}$). 

<div id="eq-4" style="font-size: 1rem;">$$ \left[ \begin{array}{c} p(w_{1}|w^{(t)}) \\ p(w_{2}|w^{(t)}) \\ p(w_{3}|w^{(t)}) \\ \vdots \\ p(w_{V}|w^{(t)}) \end{array} \right] = \frac{exp(W_{output} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{V}\tag{4} $$</div>

### Softmax is computationally very expensive

There is an issue with the vanilla Skip-Gram — softmax is computationally very expensive, as it requires scanning through the entire output embedding matrix ($W_{output}$) to compute the probability distribution of all $V$ words, where $V$ can be millions or more.

<div class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/vanilla-skip-gram-complexity.png"></div>
    <div class="col-12"><p class="image-description">Figure 3: Algorithm complexity of vanilla Skip-Gram</p></div>
</div>

Furtheremore, the normalization factor in the denominator also requires $V$ iterations. In mathematical context, the normalization factor needs to be computed for each probability p($w_{context}| w_{center}$), making the alogrithm complexity = $O(V \times V)$. However, when implemented on code, the normalization factor is computed only once and cached as a Python variable, making the alogrithm complexity = $O(V + V) \approx O(V)$. This is possible because normalization factor is the same for all words.

Due to this computational inefficiency, **softmax is not used in most implementaions of Skip-Gram**. Instead we use an alternative called *negative sampling* with <a href="">sigmoid function</a>, which rephrases the problem into a set of independent binary classification task of algorithm complexity = $O(K \, + \, 1)$, where $K$ typically has a range of $[5, 20]$.

## Skip-Gram Negative Sampling

In Skip-Gram, assuming <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic" target="_blank">stochastic gradient descent</a>, weight marices in the neural network are updated for each training sample to correctly predict output. Let's assume that the training corpus has 10,000 unique vocabs ($V$ = 10000) and the hidden layer is 300-dimensional ($N$ = 300). This means that there are 3,000,000 neurons in the output weight matrix ($W_{output}$) that need to be updated for each training sample (Notes: for the input weight matrix ($W_{input}$), only 300 neurons are updated for each training sample. This is illustrated in figure 18 of my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#weight_update" target="_blank">previous post</a>.) Since the size of the training corpus ($T$) is very large, updating 3M neurons for each training sample is unrealistic in terms of computational efficiency. Negative sampling addresses this issue by updating only a small fraction of the output weight neurons for each training sample. 

In negative sampling, $K$ <a href="neg_word">negative samples</a> are <a href="#neg_drawn">randomly drawn</a> from a <a href="noise_dist">noise distribution</a>. $K$ is a hyper-parameter that can be empirically tuned, with a typical range of $[5,\, 20]$. For each training sample, you randomly draw $K$ number of negative samples from a noise distribution, $P_n(w)$, and the model will update $(k+1) \times N$ neurons in the output weight matrix ($W_{output}$). $N$ is the dimension of the hidden layer ($h$), or the size of a word vector. $+1$ accounts for a <a href="pos_word">positive sample</a>. 

With the above assumption, if you set <code>K=9</code>, the model will update (9 + 1) * 300 = 3000 neurons, which is only 0.1% of the 3M neurons in $W_{output}$. This is computationally much faster than the original Skip-Gram, and yet maintains a good quality of word vectors.

The below figure has 3-dimensional hidden layer ($N=3$), 11 vocabs ($V=11$), and 3 negative samples ($K=3$). 

<div class="row" id="fig4">
    <div class="col"><img src="jupyter_images/neg_vs_skip.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 4: Skip-Gram model structure</p></div>
</div>

<div id="negsample" class="alert alert-info">
    <h4>Notes: Choice of $K$</h4>
    <p>The <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">paper</a> (Mikolov et al., 2013) says that K=2 ~ 5 works for large data sets, and K=5 ~ 20 for small data sets.
    </p>
</div>

### How does negative sampling work?

 With negative sampling, word vectors are no longer learned by predicting context words of a center word. Instead of using softmax to compute the $V$-dimensional probability distribution of observing an output word given an input word, $p(w_O|w_I)$, the model uses sigmoid function to learn to differentiate the actual context words (*positive*) from randomly drawn words (*negative*) from the <a href="#noise_dist">noise distribution</a> $P_n(w)$. 
 
 Assume that the center word is *"regression"*. It is likely to observe *"regression"* + {*"logistic"*, *"machine"*, *"sigmoid"*, *"supervised"*, *"neural"*} pairs, but it is unlikely to observe *"regression"* + {*"zebra"*, *"pimples"*, *"Gangnam-Style"*, *"toothpaste"*, *"idiot"*}. The model maximizes the probability $p(D=1|w,c_{pos})$ of observing positive pairs, while minimizing the probability $p(D=0|w,c_{neg})$ of observing negative pairs. The idea is that if the model can distinguish between the positive pairs ($w$, $c_{pos}$) vs negative pairs ($w$, $c_{neg}$), good word vectors will be learned. 

Negative sampling converts multi-classification task into binary-classification task. The new objective is to predict, for any given word-context pair ($w,\,c$), whether the word ($c$) is in the context window of the the center word ($w$) or not. Since the goal is to identify a given word as True (*positive*, $D=1$) or False (*negative*, $D=0$), we use sigmoid function instead of softmax function. The probability of a word ($c$) appearing within the context of the center word ($w$) can be defined as the following:

<div id="eq-5" style="font-size: 1rem;">
$$
p(D=1|w,c;\theta)=\frac{1}{1+exp(-w_{output_{(j)}} \cdot h)}
\in \mathbb{R}^{1}
\tag{5} 
$$</div>

where $c$ is the word you want to know whether it came from the context window or the noise distribution. $w$ is the input (center) word, $h$ is the hidden layer, and $\theta$ is the weight matrix passed into the model. $w_{output_{(j)}}$ is the word vector from the output weight matrix ($W_{output}$) of <a href="#fig1">figure 1</a>. 

<a href="#eq-5">Eq (5)</a> computes the probability that the given word ($c$) is a <a href="#pos_word">positive word</a> ($D=1$). It only needs to be applied $K$ + 1 times instead of $V$ times for every word in the vocabulary, because $w_j$ comes from the concatenation of true context word ($c_{pos}$) and $K$ <a href="#neg_word">negative words</a> ($W_{neg} = \{ c_{neg, j}|j=1,\cdots,K \}$):

<div id="eq-6" style="font-size: 1rem;">
$$
w_{output{(j)}} \in \{c_{pos}\} \cup W_{neg}
\tag{6} 
$$
</div>

This probability is computed $K + 1$ times to obtain a probability distribution of true context word and $K$ negative samples:

<div id="eq-7" style="font-size: 1rem;">
$$
\left[ \begin{array}{c} p(D=1|w,c_{pos}) \\ p(D=1|w,c_{neg, 1}) \\ p(D=1|w,c_{neg, 2}) \\ p(D=1|w,c_{neg, 3}) \\ \vdots \\ p(D=1|w,c_{neg, 4}) \end{array} \right] 
= 
\frac{1}{1+exp(-(\{c_{pos}\} \cup W_{neg}) \cdot h)}
\in \mathbb{R}^{K+1}\tag{7} 
$$
</div>

Compare this equation with <a href="#eq-4">eq (4)</a> — you will notice that <a href="#eq-7">eq (7)</a> is computationally much cheaper because $K$ is between 5 ~ 20, whereas $V$ can be millions. Moreover, no extra iterations are necessary to compute the normalization factor in the denominator of <a href="#eq-4">eq (4)</a>, because sigmoid function is a binary regression classifier. The algorithm complexity for probability distribution of vanilla Skip-Gram is $O(V)$, whereas negative sampling's is $O(K+1)$. This shows why negative sampling saves a significant amount of computational cost per iteration.

<div><hr></div>
<div id="fig5" class="row give-margin-inline-big-plot">
    <div class="col"><img src="jupyter_images/negative_sample_text.png"></div>
    <div class="col-12"><p class="image-description">Figure 5: Choice of negative samples (Text source: <a href="https://petrowiki.org/Drilling_induced_formation_damage">Petrowiki</a>)</p></div>
</div>

For the purpose of illustration, consider the above paragraphs. Assume that our center word ($w$) is <code>drilling</code>, window size is $3$, and the number of negative samples ($K$) is $5$. With the window size of $3$, the contexts words are: *"engineer"*, *"traditionally"*, *"designs"*, *"fluids"*, *"with"*, and *"two"*. These context words are considered as positive labels ($D$ = 1). Our current context word ($c_{pos}$) is <code>engineer</code>. We also need negative words. We randomly pick $5$-words from the <a href="#noise_dist">noise distribution</a> $P_n(w)$ of the corpus for each context word, and consider them as negative samples ($D$ = 0). For the current context word, <code>engineer</code>, the 5 randomly drawn negative words ($c_{neg}$) are: *"minimized"*, *"primary"*, *"concerns"*, *"led"*, and *"page"*. 

The idea of negative sampling is that it is more likely to observe positive word pairs ($w$, $c_{pos}$) together than negative word pairs ($w$, $c_{neg}$) together in the corpus. The model attempts to maximize the the probability of observing positive pairs $p(c_{pos}|w) \rightarrow 1$ and minimize the probability of observing negative pairs $p(c_{neg}|w) \rightarrow 0$ simultaneously by iterating through the training samples and updating the weights ($\theta$). Note that the sum of the probability distribution obtained by sigmoid function (<a href="#eq-7">eq (7)</a>) does not need to equal $1$, unlike softmax (<a href="#eq-4">eq (4)</a>).

<div id="fig6" class="row">
    <div class="col"><img src="jupyter_images/neg_opt_1.png"></div>
    <div class="col-12"><p class="image-description">Figure 6: maximizing postive pairs and minimizing negative pairs</p></div>
</div>

By the time the output probability distribution is nearly one-hot-encoded as in $iter = 4$ of the above figure, weight matrices $\theta$ are optimized and good word vectors are learned. This optimization can be done my maximizing the dot product of the input weight matrix and the hidden layer ($-w_{output_{(j)}} \cdot h$) in <a href="#">eq (333)</a>.  

<div id="negsample" class="alert alert-info">
    <h4>Notes: Drawing random negative samples</h4>
    <p>For each word-context pair ($w,\,c$), $K$ new negative samples are <a href="#neg_drawn">randomly drawn</a> from a noise distribution. In <a href="#fig5">figure 5</a>, there are 6 positive context words (<i>"engineer"</i>, <i>"traditionally"</i>, <i>"designs"</i>, <i>"fluids"</i>, <i>"with"</i>, and <i>"two"</i>) for one center word (<i>"drilling"</i>), and $K$ is 5. This means that a total of 6 * 5 = 30 word vectors are updated for each center word.</p>
</div>

<div id="noise_dist"></div>

#### What is a noise distribution $P_n(w)$?

Imagine a distribution of words based on how many times each word appeared in a corpus, $U(w)$ (this is called unigram distribution). For each word $w$, divide the number of times it appeared by a normalization factor $Z$ so that the distribution becomes a probability distribution of range $[0, 1)$ and sums up to $1$. Raise the normalized distribution to the power of $\alpha$ so that the distribution is "smoothed-out". Then this becomes your noise distribution $P_n(w)$ — normalized frequency distribution of words raised to the power of $\alpha$. Mathematically, it can be expressed as:

<div id="eq-8" style="font-size: 1rem;">
$$
P_n(w) = \left(\frac{U(w)}{Z}\right)^{\alpha}
\tag{8} 
$$</div>

Raising the unigram distribution $U(w)$ to the power of $\alpha$ has an effect of smoothing out the distribution. **It attempts to combat the imbalance between common words and rare words** by decreasing the probability of drawing common words, and increasing the probability drawing rare words. 

$\alpha$ is a hyper-parameter that can be empirically tuned. The authors of the original Word2Vec <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">paper</a> claims that the unigram distribution $U(w)$ raised to the $3/4$rd power (i.e., $U(w)^{3/4}/Z$) yielded the best result.

<div id="fig7" class="row">
    <div class="col"><img src="jupyter_images/noise_dist.png"></div>
    <div class="col-12"><p class="image-description">Figure 7: Effect of raising power of unigram distribution $U(w)$</p></div>
</div>

<div id="pos_word"></div>
#### What is a positive word $c_{pos}$?

Words that actually appear within the context window of the center word ($w$). After the model is optimized, the probability computed with <a href="#eq-5">eq (5)</a> will output $\approx$ 1 as shown in <a href="#fig6">fig 6</a>. A word vector of a center word ($w$) will be more similar to a word vector of positive word ($c_{pos}$) than of randomly drawn negative words ($c_{neg}$). This is because words that frequently appear together show strong correlation with each other.

<div id="neg_word"></div>
#### What is a negative word $c_{neg}$?

Words that are randomly drawn from a noise distribution $P_n(w)$. After the model is optimized, the probability computed with <a href="#eq-5">eq (5)</a> will output $\approx$ 0 as shown in <a href="#fig6">fig 6</a>. When training an Word2Vec model, the vocab size ($V$) easily exceeds tens of thousands. When 5 ~ 20 negative samples are randomly drawn among the vocabs, it is unlikely to observe the random word with a center word together. Therefore, once the model is optimized: $p(D=1|w,c_{neg})\approx0$.

<div id="neg_drawn"></div>
#### How are negative samples drawn?

$K$-negative samples are randomly drawn from a <a href="#noise_dist">noise distribution</a> $P_n(w)$ using someting like <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html" target="_blank">np.random.choice</a>.

In [24]:
import numpy

words = ['apple', 'bee', 'desk', 'chair']
normalized_word_frequency = [0.023, 0.12, 0.34, 0.517]

np.random.choice(words, size=20, p=normalized_word_frequency)

array(['chair', 'bee', 'chair', 'bee', 'chair', 'bee', 'chair', 'bee',
       'desk', 'bee', 'chair', 'desk', 'desk', 'desk', 'chair', 'desk',
       'chair', 'desk', 'chair', 'desk'], dtype='<U5')

<div><hr></div>
*"chair"* appeared the most from the corpus, and has the highest probability (<code>0.517</code>) of being drawn as a negative sample. As a result, *"chair"* was drawn the most. The noise distribution is raised to the power of $3/4$rd to combat the imbalance between common vs rare words, as shown in <a id="#fig7">figure 7</a>. 

## Derivation of Cost Function in Negative Sampling

Coming soon...

<div style="display: none">
    
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib notebook

# sample data generation
data = sorted(stats.norm.rvs(size=1000) + 5)

# fit normal distribution
mean, std = stats.norm.fit(data, loc=0)
pdf_norm = stats.norm.pdf(data, mean, std)

temp = np.power(data, 3/4)
temp_mean, temp_std = stats.norm.fit(temp, loc=0)
temp_pdf_norm = stats.norm.pdf(temp, temp_mean, temp_std)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
ax1.set_title('$U(w)$')
ax1.hist(temp, bins='auto', density=True)
ax1.set_xticklabels(['apple', 'desk', 'cup', 'chair', 'zebra', 'room', 'pencil', 'water', 'coin'])
ax1.plot(temp, temp_pdf_norm, label='norm')
ax2.set_title('$U(w)^{3/4}$')
ax2.hist(data, bins='auto', density=True)
ax2.plot(data, pdf_norm, label='norm')
ax2.set_xticklabels(['apple', 'desk', 'cup', 'chair', 'zebra', 'room', 'pencil', 'water', 'coin'])
</div>